In [6]:
import os
import warnings
import nibabel as nib

In [ ]:
def process_GluCEST(data_dir):
    ''' Process GluCEST
    
    
    Parameters:
    
    data_dir (str): path to directory that stores 2d CESt, B0, B1, 2d T1 maps by subject
    
    
    Returns:
    
    
    '''
    
    CESTppm = 1.3
    
    # iterate through each subject in data_dir
    for subject in os.listdir(data_dir):
        
        # get the absolute path of subject and iterate to parse each file type
        subject_path = os.path.join(data_dir, subject)
        
        # set <file_type>_path = None to later check if this subject has all necessary files
        ref_path = None
        cest_path = None
        mp2rageT1_path = None
        wassr_path = None
        b1map_path = None
        
        # parse each file type to be read later
        for dcm_dir in os.listdir(subject_path):
        
            # ref image
            if ("none" in dcm_dir):
                ref_path = os.path.join(subject_path, dcm_dir)
            # cest image
            elif ("cest" in dcm_dir):
                cest_path = os.path.join(subject_path, dcm_dir)
            # mp2rage T1 image
            elif ("mp2rage" in dcm_dir):
                mp2rageT1_path = os.path.join(subject_path, dcm_dir)
            # wassr image
            elif ("wassr" in dcm_dir):
                wassr_path = os.path.join(subject_path, dcm_dir)
            # b0 image
            elif ("b1map" in dcm_dir):
                b1map_path = os.path.join(subject_path, dcm_dir)
            # brain mask image
            # output of line 66 in Val's Sructural_processing.sh
            # brain mask applied to UNI image
            # then run through to line 97 of GluCEST_processing.sh
            # that finds the corresponding UNI slab to CEST slab.
            # find the same slab in the UNI mask, align and apply to cest slab
            elif ("-UNI-mask-er.nii.gz" in dcm_dir):
                mask_path = os.path.join(subject_path, dcm_dir)
        
        # if the subject is missing any of the above files, skip over
        if None in [ref_path, cest_path, mp2rageT1_path, wassr_path, b1map_path]:
            warnings.warn("Subject " + subject + " is missing necessary file(s).", 
                         UserWarning)
            continue
        
        # subject has all the necessary files so now read them all
        ref_info = read_ref(ref_path)
        cest_info = read_cest(cest_path)
        mp2rageT1_info = read_mp2rageT1(mp2rageT1_path)
        wassr_info = read_wassr(wassr_path)
        b1map_info = read_B1(b1map_path)
        
        # load brain mask
        brain_mask_img = nib.load(mask_path)
        
#         # calculating B0
#         b0map = mexwassr_b0map(wassr_info["WASSR_ppm_list"],
#                                wassr_info["WASSR_pos_img"],
#                                wassr_info["WASSR_neg_img"],
#                                brain_mask_img,
#                                0.005)
        
        # write dicom files for wassr b0mapm line 928-940
#         rescale_flag = 1
#         new_series = 991
#         new_description = "WASSR_B0MAP2D"
#         fnames = dicom_create2d()
            
        calc_B1map_new(b1map_info["B1_img1"], b1map_info["B1_img2"], b1map_info["B1_img3"],
                       brain_mask_img, b1map_info["alpha"])
        